In [12]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

In [13]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)

In [14]:
print(X_train[5])
print(type(X_train[1]))
print(len(X_train[5]))

[1, 778, 128, 74, 12, 630, 163, 15, 4, 1766, 2, 1051, 2, 32, 85, 156, 45, 40, 148, 139, 121, 664, 665, 10, 10, 1361, 173, 4, 749, 2, 16, 3804, 8, 4, 226, 65, 12, 43, 127, 24, 2, 10, 10]
<class 'list'>
43


In [15]:
from collections import Counter

def count_values(y_test):
    # Use Counter to count occurrences of each value in the array
    value_counts = Counter(y_test)
    
    # Print the counts
    for value, count in value_counts.items():
        print(f"{value}: {count}")

# Example usage:
#array = [1, 2, 3, 4, 1, 2, 3, 1, 2, 4, 5]
count_values(y_test)


0: 12500
1: 12500


In [16]:
## It's means that the given data is already balance data

In [17]:
# truncate and/or pad input sequences
max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(25000, 600)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [18]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary()) 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 600, 32)           160032    
                                                                 
 lstm_2 (LSTM)               (None, 100)               53200     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213333 (833.33 KB)
Trainable params: 213333 (833.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [19]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)
#model.fit(X_train, y_train, epoch=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
391/391 [==============================] - 397s 1s/step - loss: 0.4649 - accuracy: 0.7706 - val_loss: 0.3256 - val_accuracy: 0.8637
Epoch 2/10
391/391 [==============================] - 399s 1s/step - loss: 0.2957 - accuracy: 0.8803 - val_loss: 0.3064 - val_accuracy: 0.8722
Epoch 3/10
391/391 [==============================] - 398s 1s/step - loss: 0.2494 - accuracy: 0.9049 - val_loss: 0.3122 - val_accuracy: 0.8688
Epoch 4/10
391/391 [==============================] - 418s 1s/step - loss: 0.2195 - accuracy: 0.9145 - val_loss: 0.3589 - val_accuracy: 0.8598
Epoch 5/10
391/391 [==============================] - 383s 981ms/step - loss: 0.1885 - accuracy: 0.9284 - val_loss: 0.3337 - val_accuracy: 0.8746
Epoch 6/10
391/391 [==============================] - 391s 999ms/step - loss: 0.1658 - accuracy: 0.9387 - val_loss: 0.3588 - val_accuracy: 0.8672
Epoch 7/10
391/391 [==============================] - 388s 992ms/step - loss: 0.1529 - accuracy: 0.9438 - val_loss: 0.4106 - val_accurac

In [ ]:
# Model Accuracy may change due to retraining.

# LSTM for Sequence Classification with Dropout

## Recurrent neural networks like LSTM generally have the problem of overfitting.
Dropout can be applied between layers using the Dropout Keras layer. You can do this easily by adding new Dropout layers between the Embedding and LSTM layers and the LSTM and Dense output layers. For example

In [20]:
from tensorflow.keras.layers import Dropout

In [21]:
# create the model
embedding_vecor_length = 32
model1 = Sequential()
model1.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model1.add(Dropout(0.2))
model1.add(LSTM(100))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 600, 32)           160000    
                                                                 
 dropout_2 (Dropout)         (None, 600, 32)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213301 (833.21 KB)
Trainable params: 213301 (833.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [22]:
model1.fit(X_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model1.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
391/391 [==============================] - 309s 786ms/step - loss: 0.4617 - accuracy: 0.7778
Epoch 2/10
391/391 [==============================] - 315s 806ms/step - loss: 0.2828 - accuracy: 0.8866
Epoch 3/10
391/391 [==============================] - 330s 844ms/step - loss: 0.2702 - accuracy: 0.8928
Epoch 4/10
391/391 [==============================] - 328s 838ms/step - loss: 0.2176 - accuracy: 0.9168
Epoch 5/10
391/391 [==============================] - 338s 864ms/step - loss: 0.2150 - accuracy: 0.9162
Epoch 6/10
391/391 [==============================] - 352s 899ms/step - loss: 0.1986 - accuracy: 0.9248
Epoch 7/10
391/391 [==============================] - 346s 886ms/step - loss: 0.1742 - accuracy: 0.9345
Epoch 8/10
391/391 [==============================] - 349s 892ms/step - loss: 0.1550 - accuracy: 0.9422
Epoch 9/10
391/391 [==============================] - 351s 899ms/step - loss: 0.1457 - accuracy: 0.9452
Epoch 10/10
391/391 [==============================] - 350s 896m

In [ ]:
# Model Accuracy may change due to retraining.

We apply Convolution Nural Network with the sequence learning

In [23]:
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [24]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 600, 32)           160000    
                                                                 
 conv1d (Conv1D)             (None, 600, 32)           3104      
                                                                 
 max_pooling1d (MaxPooling1  (None, 300, 32)           0         
 D)                                                              
                                                                 
 lstm_4 (LSTM)               (None, 100)               53200     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 216405 (845.33 KB)
Trainable params: 216405 (845.33 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [26]:
model.fit(X_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
391/391 [==============================] - 211s 534ms/step - loss: 0.4189 - accuracy: 0.7954
Epoch 2/10
391/391 [==============================] - 209s 535ms/step - loss: 0.2459 - accuracy: 0.9034
Epoch 3/10
391/391 [==============================] - 213s 545ms/step - loss: 0.2042 - accuracy: 0.9233
Epoch 4/10
391/391 [==============================] - 211s 540ms/step - loss: 0.1653 - accuracy: 0.9398
Epoch 5/10
391/391 [==============================] - 207s 531ms/step - loss: 0.1407 - accuracy: 0.9502
Epoch 6/10
391/391 [==============================] - 202s 516ms/step - loss: 0.1083 - accuracy: 0.9638
Epoch 7/10
391/391 [==============================] - 199s 510ms/step - loss: 0.0926 - accuracy: 0.9700
Epoch 8/10
391/391 [==============================] - 205s 523ms/step - loss: 0.0745 - accuracy: 0.9775
Epoch 9/10
391/391 [==============================] - 200s 512ms/step - loss: 0.0637 - accuracy: 0.9802
Epoch 10/10
391/391 [==============================] - 195s 500m

In [ ]:
# Model Accuracy may change due to retraining.

In [27]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, return_sequences=True))  # LSTM with return_sequences=True
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [28]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 600, 32)           160000    
                                                                 
 lstm_5 (LSTM)               (None, 600, 100)          53200     
                                                                 
 conv1d_1 (Conv1D)           (None, 600, 32)           9632      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 300, 32)           0         
 g1D)                                                            
                                                                 
 lstm_6 (LSTM)               (None, 100)               53200     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                      

In [29]:
# Here we Probem with parameter,we see before that (None,1) but here we see that (None,300,1)

In [30]:
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
391/391 [==============================] - 583s 1s/step - loss: 0.4305 - accuracy: 0.7925
Epoch 2/3
391/391 [==============================] - 627s 2s/step - loss: 0.2663 - accuracy: 0.8945
Epoch 3/3
391/391 [==============================] - 599s 2s/step - loss: 0.2198 - accuracy: 0.9147
Accuracy: 88.84%
